<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#imports" data-toc-modified-id="imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>imports</a></span></li><li><span><a href="#methods" data-toc-modified-id="methods-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>methods</a></span></li><li><span><a href="#Data-processing" data-toc-modified-id="Data-processing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data processing</a></span><ul class="toc-item"><li><span><a href="#use-working-directory" data-toc-modified-id="use-working-directory-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>use working directory</a></span><ul class="toc-item"><li><span><a href="#the-issue-has-been-resolved-in-the-beginning-of-the-model-notebook" data-toc-modified-id="the-issue-has-been-resolved-in-the-beginning-of-the-model-notebook-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>the issue has been resolved in the beginning of the model notebook</a></span></li></ul></li></ul></li></ul></div>

# imports

In [3]:
import keras
import numpy as np
import datetime

from keras.preprocessing.image import ImageDataGenerator
from collections import defaultdict
from glob import glob
from random import choice, sample

# Florencia's imports
import tensorflow
from tensorflow.keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D,Activation, Dropout, BatchNormalization,  GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Subtract,Multiply
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from skimage.io import imshow
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import cv2
import os
from os import listdir
from tqdm import tqdm
from tensorflow.keras.applications.resnet50 import preprocess_input
from random import sample, choice

from tensorflow.keras import layers
from tensorflow.keras import preprocessing
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K


# methods

In [4]:

def euclidean_distance(vects): # Euc_dist between 2 vectors
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes): # ?
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

from keras.preprocessing.image import ImageDataGenerator
from random import randint
from random import uniform as rfloat
def random_aug(img, img_gen):
    transform_parameters = { # full transform params list @ keras image preprocessing documentation
         'shear':randint(-8,8),
        'zx':rfloat(0.9,1.0), "zy":rfloat(0.9,1.0), 
        'theta':randint(-20,20), "flip_horizontal":bool(random.getrandbits(1))}
#     img_gen = ImageDataGenerator()
    return img_gen.apply_transform(img, transform_parameters)

def read_img(path): 
    img = cv2.imread(path)
    img = np.array(img).astype(np.float)
    return preprocess_input(img)
def paths_to_arrays(paths_list):
    out = []
    for path in paths_list:
        out.append(read_img(path))
    return out

# Data processing

## use working directory

In [5]:
# my apologies for the hardcoding
dir_path = '/Users/mark/Documents/flatiron/projects/love/teaching_robots_to_swipe/'
train_part = 'UTKface/train/'
val_part = 'UTKface/validation/'
test_part = 'UTKface/test/'
# no overlap between train, val, test imgs
train_path = dir_path+train_part # 2008
val_path = dir_path+val_part # 95
test_path = dir_path+test_part # 155, distinct from val

In [6]:
# get all files
trains = glob(train_path + '*.jpg')
tests = glob(test_path + '*.jpg')
vals = glob(val_path + '*.jpg')
# trains

In [7]:
print(trains[0])
print(tests[0])
print(vals[0])

/Users/mark/Documents/flatiron/projects/love/teaching_robots_to_swipe/UTKface/train/26_1_1_20170103181931657.jpg.chip.jpg
/Users/mark/Documents/flatiron/projects/love/teaching_robots_to_swipe/UTKface/test/25_0_2_20170104192902767.jpg.chip.jpg
/Users/mark/Documents/flatiron/projects/love/teaching_robots_to_swipe/UTKface/validation/35_1_0_20170103163453110.jpg.chip.jpg


In [8]:
print(len(trains), len(tests), len(vals))

2008 155 95


In [9]:
train_data = paths_to_arrays(trains)
test_data = paths_to_arrays(tests)
val_data = paths_to_arrays(vals)

In [10]:
np.shape(train_data)

(2008, 200, 200, 3)

In [11]:
np.shape(test_data) #why is test giving a vector instead of tensor? should be RGB

(155,)

In [12]:
np.shape(val_data) # they're all lists of arrays

(95, 200, 200, 3)

In [13]:
np.shape(test_data) # a list of vectors...

(155,)

In [14]:
np.shape(test_data[0]) # no, they're tensors. what foul sorcery is this?

(200, 200, 3)

### the issue has been resolved in the beginning of the model notebook